# This is an implementation of a neural network using only raw python and numpy
* The network is trained on the cats and dogs dataset
* The network has 4 hidden layers with 20 neurons each

Coding the neural network from scratch is a by no means a good idea for real world applications. However, it is a good way to understand the inner workings of a neural network. This notebook is a good starting point for anyone who wants to learn about neural networks.


In [4]:
import numpy as np

In [13]:
np.random.seed(0)

X = [[1,2,3,2.5],
          [2.0,5.0,-1.0,2.0],
          [-1.5,2.7,3.3,-0.8]]



class Layer_Dense:

    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

layer1 = Layer_Dense(4, 5)
layer2 = Layer_Dense(5, 2)
